In [1]:
# Update recommender for additional features, and to accommodte reaction_recommender
import editdistance
import libsbml
import numpy as np
import operator
import os
import compress_pickle
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import constants as cn
from AMAS import recommender
from AMAS import tools

In [2]:
with open(os.path.join(cn.REF_DIR, 'chebi_shortened_formula_comp.lzma'), 'rb') as f:
  ref_shortened_chebi_to_formula = compress_pickle.load(f)

In [3]:
one_biomd = 'BIOMD0000000190.xml'
one_biomd_fpath = os.path.join(BIOMODEL_DIR, one_biomd)

In [4]:
# First, sending a libsbml.SBMLDocument class instance
reader = libsbml.SBMLReader()
document = reader.readSBML(one_biomd_fpath)
# 
print("File Type of document is: %s \n" % str(type(document)))
recom = recommender.Recommender(document)
# checking if model was loaded successfully
print(recom.species.names)
print()
print(recom.reactions.reaction_components)

File Type of document is: <class 'libsbml.SBMLDocument'> 

{'SAM': 'S-adenosyl-L-methionine', 'A': 'S-adenosylmethioninamine', 'P': 'Putrescine', 'S': 'Spermine', 'D': 'Spermidine', 'aS': 'N1-Acetylspermine', 'aD': 'N1-Acetylspermidine', 'Met': 'Methionine', 'ORN': 'L-Ornithine', 'AcCoA': 'Acetyl-CoA', 'CoA': 'CoA'}

{'ODC': ['P', 'ORN'], 'SAMdc': ['A', 'SAM'], 'SSAT_for_S': ['S', 'AcCoA', 'aS', 'CoA'], 'SSAT_for_D': ['D', 'AcCoA', 'CoA', 'aD'], 'PAO_for_aD': ['aD', 'P'], 'PAO_for_aS': ['D', 'aS'], 'SpdS': ['D', 'A', 'P'], 'SpmS': ['D', 'A', 'S'], 'MAT': ['SAM', 'Met'], 'VCoA': ['AcCoA', 'CoA'], 'VacCoA': ['AcCoA', 'CoA'], 'P_efflux': ['P'], 'aD_efflux': ['aD']}


In [5]:
# def updateSpeciesWithRecommendation(inp_recom):
#   """
#   Update species_annotation class using
#   Recommendation namedtuple.
  
#   self.candidates is a sorted list of tuples,
#   (chebi_id: match_score)
#   self.formula is a unsorted list of unique formulas
  
#   Parameters
#   ----------
#   inp_recom: Recommendation
#      Result of recom.getSpeciesAnnotation
  
#   Returns
#   -------
#   None
#   """
#   recom.species.candidates.update({inp_recom.id: inp_recom.candidates})
#   formulas2update = list(set([ref_shortened_chebi_to_formula[val[0]] for val in inp_recom.candidates]))
#   recom.species.formula.update({inp_recom.id: formulas2update})
#   return None

In [6]:
# updateSpeciesWithRecommendation(inp_recom=species_recom)

In [7]:
print(recom.species.candidates)
print(recom.species.formula)

{}
{}


In [12]:
# Updating reaction annotation part
recom = recommender.Recommender(libsbml_fpath=one_biomd_fpath)
pred_id = 'ODC'
update=True

In [13]:
specs2predict = recom.reactions.reaction_components[pred_id]
spec_results = recom.getSpeciesListAnnotation(pred_ids=specs2predict)
# based on the function above; need to recreate it. 
pred_formulas = dict()
for one_recom in spec_results:
  chebis = [val[0] for val in one_recom.candidates]
  forms = list(set([cn.ref_chebi2formula[k] \
           for k in chebis if k in cn.ref_chebi2formula.keys()]))
  pred_formulas[one_recom.id] = forms
#
pred_reaction = recom.reactions.predictAnnotation(inp_spec_dict=pred_formulas,
                                                 inp_reac_list=[pred_id],
                                                 update=update)

In [14]:
pred_reaction

{'ODC': [('RHEA:28830', 1.0), ('RHEA:22967', 0.5), ('RHEA:59051', 0.5)]}

In [16]:
{one_k: len(pred_reaction[one_k]) for one_k in pred_reaction}

{'ODC': 3}